In [1]:
import os
import geopandas as gpd
import pandas as pd
from sklearn.neighbors import BallTree
import h3
import zipfile
from shapely.geometry import Point
from uszipcode import SearchEngine
import numpy as np

os.environ['USE_PYGEOS'] = '0'


def get_centroid(ziploc):
    def get_point_from_zip(code):
        zipcode = engine.by_zipcode(code)
        return Point(zipcode.lat, zipcode.lng)

    if ziploc == "28":
        return get_point_from_zip(39762)
    elif len(ziploc) == 15:
        lat_lon = h3.h3_to_geo(ziploc)
        return Point(lat_lon[0], lat_lon[1])
    elif len(ziploc) == 5:
        return get_point_from_zip(ziploc)

    return None


def get_point(lat, lng):
    return Point(float(lat), float(lng))


def get_nearest(src_points, candidates, k_neighbors=1):
    """Find nearest neighbors for all source points from a set of candidate points"""

    # Create tree from the candidate points
    tree = BallTree(candidates, leaf_size=15, metric='haversine')

    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)

    # Transpose to get distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()

    # Get closest indices and distances (i.e. array at index 0)
    # note: for the second closest points, you would take index 1, etc.
    closest = indices[0]
    closest_dist = distances[0]

    # Return indices and distances
    return (closest, closest_dist)


def geo_radians(geom):
    try:
        geom.y * np.pi / 180, geom.x * np.pi / 180
    except Exception as e:
        print(f"Ex: {e}")
        print(geom)
        return None


def nearest_neighbor(left_gdf, right_gdf, return_dist=False):
    """
    For each point in left_gdf, find closest point in right GeoDataFrame and return them.

    NOTICE: Assumes that the input Points are in WGS84 projection (lat/lon).
    """

    left_geom_col = 'centroid'
    right_geom_col = 'centroid'  # right_gdf.geometry.name

    # Ensure that index in right gdf is formed of sequential numbers
    right = right_gdf.copy().reset_index(drop=True)

    # Parse coordinates from points and insert them into a numpy array as RADIANS
    # Notice: should be in Lat/Lon format
    left_radians = np.array(
        left_gdf[left_geom_col].apply(lambda geom: (geom.y * np.pi / 180, geom.x * np.pi / 180)).to_list())
    right_radians = np.array(
        right[right_geom_col].apply(lambda geom: (geom.y * np.pi / 180, geom.x * np.pi / 180)).to_list())
    # df_locations.apply(lambda x: geo_radians(x['centroid']), axis=1).to_list()

    # Find the nearest points
    # -----------------------
    # closest ==> index in right_gdf that corresponds to the closest point
    # dist ==> distance between the nearest neighbors (in meters)

    closest, dist = get_nearest(src_points=left_radians, candidates=right_radians)

    # Return points from right GeoDataFrame that are closest to points in left GeoDataFrame
    closest_points = right.loc[closest]

    # Ensure that the index corresponds the one in left_gdf
    closest_points = closest_points.reset_index(drop=True)

    # Add distance if requested
    if return_dist:
        # Convert to meters from radians
        earth_radius = 6371000  # meters
        closest_points['distance'] = dist * earth_radius

    return closest_points


/Users/chethan.umesha/Work/personal/autoetl/geotl/.venv/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/rp/1ymdcv2s3193skqbjrqv1xmr0000gn/T/ipykernel_39150/1014007051.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from 

In [2]:
# Data Source 1
# CSV
# ms_hinds_locations = pd.read_csv("data/ms_hinds_locations.csv")
ms_hinds_locations = gpd.read_file("../data/ms_hinds_locations.csv")
print(ms_hinds_locations.columns)

ms_hinds_locations.head(2)

# Data Source 4

zf = zipfile.ZipFile("../data/ms_hinds_buildings.geojson.zip")
# df = pd.read_csv(zf)
# dfcsv = gpd.read_file("data/ms_hinds_buildings.geojson/ms_hinds_buildings_join_table.csv")
# dfjson = gpd.read_file("data/ms_hinds_buildings.geojson/ms_hinds_buildings.json")

ms_hinds_buildings = [gpd.read_file(zf.open(text_file.filename)) for text_file in zf.infolist() if
                      text_file.filename.endswith('.json')]
df_ms_buildings = ms_hinds_buildings[0]

df_locations = ms_hinds_locations
df_locations.crs = 'epsg:4326'
df_locations = df_locations.to_crs({'init': 'epsg:4326'})

df_ms_buildings = df_ms_buildings.to_crs(crs={'init': 'epsg:4326'})
df_ms_buildings['centroid'] = df_ms_buildings.centroid

engine = SearchEngine(simple_or_comprehensive=SearchEngine.SimpleOrComprehensiveArgEnum.comprehensive)

df_locations['f_lat'] = df_locations['f_lat'].astype(float)
df_locations['f_lon'] = df_locations['f_lon'].astype(float)
df_locations['centroid'] = df_locations.apply(
    lambda x: get_centroid(x['f_ziploc']) if pd.isnull(x['f_lat']) or pd.isnull(x['f_lon']) else get_point(
        x['f_lat'], x['f_lon']),
    axis=1)

# Find closest public transport stop for each building and get also the distance based on haversine distance
# Note: haversine distance which is implemented here is a bit slower than using e.g. 'euclidean' metric
# but useful as we get the distance between points in meters

df_ms_buildings.isna().sum()
df_locations.isna().sum()

# And the result looks like ..
closest_building = nearest_neighbor(df_ms_buildings, df_locations, return_dist=True)

# Rename the geometry of closest stops gdf so that we can easily identify it
closest_building = closest_building.rename(columns={'centroid': 'closest_centroid', 'geometry': 'close_geometry'})

# Merge the datasets by index (for this, it is good to use '.join()' -function)
buildings = df_ms_buildings.join(closest_building)
# closest_building[['geometry']].isna().sum()

Index(['source', 'parcel_id', 'parcel_lat', 'parcel_lon',
       'parcel_building_footprint', 'parcel_building_count', 'poi_lat',
       'poi_lon', 'secondary_lat', 'secondary_lon', 'lob_addr1', 'lob_lat',
       'lob_lon', 'lob_zipcode', 'f_ziploc', 'f_lat', 'f_lon', 'f_city',
       'primary_loc_id', 'f_addr1', 'f_unit', 'geometry'],
      dtype='object')


/Users/chethan.umesha/Work/personal/autoetl/geotl/.venv/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/var/folders/rp/1ymdcv2s3193skqbjrqv1xmr0000gn/T/ipykernel_39150/2542321353.py:25: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_ms_buildings['centroid'] = df_ms_buildings.centroid


In [4]:
# Now we should have exactly the same number of closest_building as we have buildings
print(len(closest_building), '==', len(df_ms_buildings['centroid']))


136954 == 136954


In [9]:
# !pip install geopandas

In [11]:
buildings

,ed_str_uuid,ed_bld_uuid,ed_geoid,ed_lat,ed_lon,ed_bldg_footprint_sqft,ed_source,ed_source_date,geometry,centroid,...,f_ziploc,f_lat,f_lon,f_city,primary_loc_id,f_addr1,f_unit,close_geometry,closest_centroid,distance
0,d4f432d1-b9b7-11ec-9de6-3417ebd2351c,69270684-beeb-11ec-9ed5-4439c4546725,28049,32.063243,-90.373405,1326.936366,Aerial Imagery,06/16/2020,"MULTIPOLYGON (((-90.37332 32.06327, -90.37335 ...",POINT (-90.37340 32.06324),...,39175,32.13367,-90.759415,Utica,18074@@OLD PORT GIBSON@RD@&&39175,18074 OLD PORT GIBSON RD,nan,None,POINT (32.134 -90.759),1.352719e+07
1,63779344-b9b6-11ec-969a-3417ebd2351c,71d51494-bee9-11ec-8398-4439c4546725,28049,32.234298,-90.254389,1598.433061,Aerial Imagery,06/13/2020,"MULTIPOLYGON (((-90.25432 32.23435, -90.25432 ...",POINT (-90.25439 32.23430),...,39175,32.13367,-90.759415,Utica,18074@@OLD PORT GIBSON@RD@&&39175,18074 OLD PORT GIBSON RD,nan,None,POINT (32.134 -90.759),1.354635e+07
2,32806cf3-b9b1-11ec-b54e-3417ebd2351c,77ed2261-becf-11ec-9644-4439c4546725,28049,32.086143,-90.302760,2439.108305,Aerial Imagery,06/13/2020,"MULTIPOLYGON (((-90.30271 32.08625, -90.30268 ...",POINT (-90.30276 32.08614),...,39175,32.13367,-90.759415,Utica,18074@@OLD PORT GIBSON@RD@&&39175,18074 OLD PORT GIBSON RD,nan,None,POINT (32.134 -90.759),1.352982e+07
3,32806cf1-b9b1-11ec-b890-3417ebd2351c,77af560f-becf-11ec-9ec4-4439c4546725,28049,32.086371,-90.308784,38341.355952,Aerial Imagery,06/13/2020,"MULTIPOLYGON (((-90.30893 32.08653, -90.30868 ...",POINT (-90.30881 32.08611),...,39175,32.13367,-90.759415,Utica,18074@@OLD PORT GIBSON@RD@&&39175,18074 OLD PORT GIBSON RD,nan,None,POINT (32.134 -90.759),1.352981e+07
4,d8e988c2-b9b7-11ec-8646-3417ebd2351c,f9cdedd8-beea-11ec-a92a-4439c4546725,28049,32.088250,-90.478211,255.556931,Aerial Imagery,06/13/2020,"MULTIPOLYGON (((-90.47820 32.08828, -90.47820 ...",POINT (-90.47821 32.08825),...,39175,32.13367,-90.759415,Utica,18074@@OLD PORT GIBSON@RD@&&39175,18074 OLD PORT GIBSON RD,nan,None,POINT (32.134 -90.759),1.352984e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136949,1aa92534-b9b3-11ec-8dd7-3417ebd2351c,53b6df35-bee3-11ec-8039-4439c4546725,28049,32.304577,-90.196741,10485.373846,Aerial Imagery,06/13/2020,"MULTIPOLYGON (((-90.19678 32.30477, -90.19681 ...",POINT (-90.19671 32.30468),...,39175,32.13367,-90.759415,Utica,18074@@OLD PORT GIBSON@RD@&&39175,18074 OLD PORT GIBSON RD,nan,None,POINT (32.134 -90.759),1.355425e+07
136950,1ae4a7a1-b9b3-11ec-bde7-3417ebd2351c,54b6c301-bee3-11ec-b9d2-4439c4546725,28049,32.308137,-90.185608,4304.012357,Aerial Imagery,06/13/2020,"MULTIPOLYGON (((-90.18545 32.30810, -90.18548 ...",POINT (-90.18553 32.30813),...,39175,32.13367,-90.759415,Utica,18074@@OLD PORT GIBSON@RD@&&39175,18074 OLD PORT GIBSON RD,nan,None,POINT (32.134 -90.759),1.355465e+07
136951,91d44e5a-b9b2-11ec-b870-3417ebd2351c,a43cc738-bed9-11ec-9e7c-4439c4546725,28049,32.352175,-90.125144,5143.425773,Aerial Imagery,06/13/2020,"MULTIPOLYGON (((-90.12517 32.35215, -90.12505 ...",POINT (-90.12515 32.35207),...,39175,32.13367,-90.759415,Utica,18074@@OLD PORT GIBSON@RD@&&39175,18074 OLD PORT GIBSON RD,nan,None,POINT (32.134 -90.759),1.355961e+07
136952,6414e4b6-b9b6-11ec-a9a2-3417ebd2351c,71fe6e89-bee9-11ec-8f1b-4439c4546725,28049,32.258642,-90.213553,2664.055032,Lidar,06/01/2016,"MULTIPOLYGON (((-90.21354 32.25857, -90.21362 ...",POINT (-90.21355 32.25863),...,39175,32.13367,-90.759415,Utica,18074@@OLD PORT GIBSON@RD@&&39175,18074 OLD PORT GIBSON RD,nan,None,POINT (32.134 -90.759),1.354911e+07


In [15]:
# df_ms_buildings.to_file("out/df_ms_buildings.geojson", driver='GeoJSON')
buildings.to_csv("df_ms_buildings.csv")

In [18]:
buildings.columns

Index(['ed_str_uuid', 'ed_bld_uuid', 'ed_geoid', 'ed_lat', 'ed_lon',
       'ed_bldg_footprint_sqft', 'ed_source', 'ed_source_date', 'geometry',
       'centroid', 'source', 'parcel_id', 'parcel_lat', 'parcel_lon',
       'parcel_building_footprint', 'parcel_building_count', 'poi_lat',
       'poi_lon', 'secondary_lat', 'secondary_lon', 'lob_addr1', 'lob_lat',
       'lob_lon', 'lob_zipcode', 'f_ziploc', 'f_lat', 'f_lon', 'f_city',
       'primary_loc_id', 'f_addr1', 'f_unit', 'close_geometry',
       'closest_centroid', 'distance'],
      dtype='object')

In [22]:
buildings[['ed_lat', 'ed_lon', 'closest_centroid', 'geometry']]

,ed_lat,ed_lon,closest_centroid,geometry
0,32.063243,-90.373405,POINT (32.134 -90.759),"MULTIPOLYGON (((-90.37332 32.06327, -90.37335 ..."
1,32.234298,-90.254389,POINT (32.134 -90.759),"MULTIPOLYGON (((-90.25432 32.23435, -90.25432 ..."
2,32.086143,-90.302760,POINT (32.134 -90.759),"MULTIPOLYGON (((-90.30271 32.08625, -90.30268 ..."
3,32.086371,-90.308784,POINT (32.134 -90.759),"MULTIPOLYGON (((-90.30893 32.08653, -90.30868 ..."
4,32.088250,-90.478211,POINT (32.134 -90.759),"MULTIPOLYGON (((-90.47820 32.08828, -90.47820 ..."
...,...,...,...,...
136949,32.304577,-90.196741,POINT (32.134 -90.759),"MULTIPOLYGON (((-90.19678 32.30477, -90.19681 ..."
136950,32.308137,-90.185608,POINT (32.134 -90.759),"MULTIPOLYGON (((-90.18545 32.30810, -90.18548 ..."
136951,32.352175,-90.125144,POINT (32.134 -90.759),"MULTIPOLYGON (((-90.12517 32.35215, -90.12505 ..."
136952,32.258642,-90.213553,POINT (32.134 -90.759),"MULTIPOLYGON (((-90.21354 32.25857, -90.21362 ..."


In [24]:
df_ms_buildings.columns

Index(['ed_str_uuid', 'ed_bld_uuid', 'ed_geoid', 'ed_lat', 'ed_lon',
       'ed_bldg_footprint_sqft', 'ed_source', 'ed_source_date', 'geometry',
       'centroid'],
      dtype='object')

In [ ]:
from keplergl import KeplerGl
df = buildings[['ed_lat', 'ed_lon', 'geometry']]
my_map = KeplerGl(data={"building_data": df}, height=600)
my_map

In [25]:
from keplergl import KeplerGl
df = buildings[['ed_lat', 'ed_lon', 'geometry']]
my_map = KeplerGl(data={"building_data": df}, height=600)
my_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'building_data':            ed_lat     ed_lon  \
0       32.063243 -90.373405   
1       32.234…

In [26]:
# from keplergl import KeplerGl

# map_1 = KeplerGl()
# map_1.add_data(buildings, name='data')
# map_1._repr_html_(read_only=True)